In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#12월 일사량 누적, 온도, 풍속 평균 그래프 시각화
# --------------------------
# 1. EPW 파일 불러오기
# --------------------------
epw_path = "../data/KOR_KG_Seoul-Seongnam.AP.471110_TMYx.2004-2018/KOR_KG_Seoul-Seongnam.AP.471110_TMYx.2004-2018.epw"  # <- 여기에 EPW 파일 경로 입력
df = pd.read_csv(epw_path, skiprows=8, header=None)

# --------------------------
# 2. 날짜 및 시간 처리
# --------------------------
df['Month'] = df[1]
df['Day'] = df[2]
df['Hour'] = df[3] - 1  # EPW의 시간은 1~24로 표기 → 0~23으로 보정

# 날짜 컬럼 추가 (선택적, 타임스탬프 만들기용)
df['Datetime'] = pd.to_datetime({
    'year': 2023,
    'month': df['Month'],
    'day': df['Day'],
    'hour': df['Hour']
}, errors='coerce')

In [3]:
# 1. 주요 기상 변수 추출
df['GHI_kWh'] = df[13] / 1000
df['Temp'] = df[6]

# 2. 데이터 필터링 (2010년 12월 데이터)
dec_2010 = df[(df[0] == 2010) & (df['Month'] == 12)].copy()

# 3. 시간 필터링 (00시~23시 전체 사용)
# → Hour 컬럼 그대로 사용 가능
hourly_avg_filtered = dec_2010.groupby('Hour')[['GHI_kWh', 'Temp']].mean()

# 4. X축 Label
hour_labels = [f"{hour:02d}:00" for hour in hourly_avg_filtered.index]

# 5. 그래프 시각화
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Bar plot (Radiation)
fig.add_trace(
    go.Bar(
        x=hour_labels,
        y=hourly_avg_filtered['GHI_kWh'],
        name='Radiation (kWh/m²)',
        marker_color='gray',
        opacity=0.8  # Bar 투명도 추가
    ),
    secondary_y=False,
)


# Temperature (Temp → 점+선)
fig.add_trace(
    go.Scatter(
        x=hour_labels,
        y=hourly_avg_filtered['Temp'],
        name='Temperature (°C)',
        mode='lines+markers',
        line=dict(color='skyblue', width=3)  # 라인 두께 4로 설정
    ),
    secondary_y=True,
)

# Layout 설정
fig.update_layout(
    legend=dict(
        x=0.85,
        y=0.98,
        xanchor='left',
        yanchor='top',
        font=dict(family="Noto Sans KR Medium", size=20, color="black"),
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="black",
        borderwidth=0
    ),
    margin=dict(l=80, r=80, t=120, b=120),
    plot_bgcolor='white',
    paper_bgcolor='whitesmoke'
)

# X축 설정
fig.update_xaxes(
    title_text="Time",
    type='category',
    tickangle=0,
    showgrid=True,
    title_font=dict(size=20, family="Noto Sans KR Medium", color="black"),
    tickfont=dict(size=15, family="Noto Sans KR Medium", color="black")  # 글자 크기 조절 추가
)

# Y축 Radiation
fig.update_yaxes(
    title_text="Radiation (kWh/m²)",
    title_font=dict(size=20, family="Noto Sans KR Medium", color="black"),
    secondary_y=False,
    range=[0, 0.5],
    showgrid=True,
    gridcolor='rgba(0,0,0,0.5)',
    gridwidth=1,
    tickvals=[round(x * 0.05, 8) for x in range(11)],
    ticktext=[f"{round(x * 0.05, 2):.2f}" for x in range(11)],
    tickmode='array'
)

# Y축 Temp → grid 표시 안함
fig.update_yaxes(
    title_text="Temperature (°C)",
    title_font=dict(size=20, family="Noto Sans KR Medium", color="black"),
    secondary_y=True,
    range=[-4, 2],
    showgrid=False   # grid 표시 안되게 설정
)

# Show the plot
fig.show()

# Save as HTML
fig.write_html("Data Analysis.html")

In [4]:
# 1. 주요 기상 변수 추출
df['GHI_kWh'] = df[13] / 1000
df['Temp'] = df[6]

# 2. 데이터 필터링 (2010년 12월 데이터)
dec_2010 = df[(df[0] == 2010) & (df['Month'] == 12)].copy()

# 3. 시간 필터링 (06시~18시 사용)
hourly_avg_filtered = dec_2010[(dec_2010['Hour'] >= 6) & (dec_2010['Hour'] <= 18)].groupby('Hour')[['GHI_kWh', 'Temp']].mean()

# 4. X축 Label
hour_labels = [f"{hour:02d}:00" for hour in hourly_avg_filtered.index]

# 5. 그래프 시각화
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Bar plot (Radiation)
fig.add_trace(
    go.Bar(
        x=hour_labels,
        y=hourly_avg_filtered['GHI_kWh'],
        name='Radiation (kWh/m²)',
        marker_color='gray',
        opacity=0.8  # Bar 투명도 추가
    ),
    secondary_y=False,
)

# Temperature (Temp → 점+선)
fig.add_trace(
    go.Scatter(
        x=hour_labels,
        y=hourly_avg_filtered['Temp'],
        name='Temperature (°C)',
        mode='lines+markers',
        line=dict(color='skyblue', width=3)  # 라인 두께 3로 설정
    ),
    secondary_y=True,
)

# Layout 설정
fig.update_layout(
    legend=dict(
        x=0.85,
        y=0.98,
        xanchor='left',
        yanchor='top',
        font=dict(family="Noto Sans KR Medium", size=20, color="black"),
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="black",
        borderwidth=0
    ),
    margin=dict(l=80, r=80, t=120, b=120),
    plot_bgcolor='white',
    paper_bgcolor='whitesmoke'
)

# X축 설정
fig.update_xaxes(
    title_text="Time",
    type='category',
    tickangle=0,
    showgrid=True,
    title_font=dict(size=20, family="Noto Sans KR Medium", color="black"),
    tickfont=dict(size=18, family="Noto Sans KR Medium", color="black")  # 글자 크기 조절 추가
)

# Y축 Radiation
fig.update_yaxes(
    title_text="Radiation (kWh/m²)",
    title_font=dict(size=20, family="Noto Sans KR Medium", color="black"),
    tickfont=dict(size=15, family="Noto Sans KR Medium", color="black"),  # 여기 추가!
    secondary_y=False,
    range=[0, 0.5],
    showgrid=True,
    gridcolor='rgba(0,0,0,0.5)',
    gridwidth=1,
    tickvals=[round(x * 0.05, 8) for x in range(11)],
    ticktext=[f"{round(x * 0.05, 2):.2f}" for x in range(11)],
    tickmode='array'
)

# Y축 Temp → grid 표시 안함
fig.update_yaxes(
    title_text="Temperature (°C)",
    title_font=dict(size=20, family="Noto Sans KR Medium", color="black"),
    tickfont=dict(size=15, family="Noto Sans KR Medium", color="black"),  # 여기 추가!
    secondary_y=True,
    range=[-4, 2],
    showgrid=False
)

# Show the plot
fig.show()

# Save as HTML
fig.write_html("Data Analysis.html")
